In [70]:
import pandas as pd
import numpy as np
from numpy import random
import random

### GA Parameter

In [71]:
n_population = 10
interation = 100
n_elitism = 2

### INITIALIZATION


#### Parameter

In [72]:
# number of part
n = 4
# number of machines
m = 3
# range for part n
P = np.arange(1, n + 1)
# range for machine m
M = np.arange(1, m + 1)
#number of operations for each part
Q = np.array([2, 3, 2, 3])
# max of operations
MaxOps = np.max(Q)
# range for operations
O = np.arange(1, MaxOps + 1)
# Processing time of operation 𝑗 of part 𝑝 on machine 𝑘
Tp = np.random.randint(1, 10, size=(n, MaxOps, m))

In [85]:
Tp

array([[[1, 2, 8],
        [4, 8, 7],
        [6, 5, 6]],

       [[1, 3, 5],
        [7, 1, 2],
        [3, 7, 4]],

       [[5, 3, 8],
        [3, 5, 7],
        [3, 3, 2]],

       [[3, 4, 2],
        [6, 7, 2],
        [7, 8, 4]]])

### Variable

##### Time Variable JSSP

In [73]:
# time variable
starting_time_of_operation = np.zeros((n, MaxOps, m))
completion_time_of_operation = np.zeros((n, MaxOps, m))
completion_time_of_processing = np.zeros((n, MaxOps, m))

#### Assembly Variable 

In [74]:
# assembly variable
# part_in_position = np.zeros()
starting_time_of_part = np.zeros((n, m))
completion_time_of_part =   np.zeros((n, m))
inventory_time_of_part = np.zeros((n, m))

## SOLUTION PRESENTATION


### Machine INIT


In [75]:
machine_init = np.random.permutation(np.arange(1, ((MaxOps*m)+1)))

### Part INIT


In [76]:
part_init = np.random.permutation(np.arange(1, n+1))

### Assembly INIT


In [77]:
assembly_init = np.random.permutation(np.arange(1, m+1))

### Full Array 

In [78]:
init = np.concatenate((machine_init, part_init, assembly_init), axis=0)

### Create pop

In [79]:
pop = []

for i in range(1, n_population + 1):
    machine_init = np.random.permutation(np.arange(1, (MaxOps * m) + 1))
    part_init = np.random.permutation(np.arange(1, n + 1))
    assembly_init = np.random.permutation(np.arange(1, n + 1))
    init = np.concatenate((machine_init, part_init, assembly_init), axis=0)
    
    pop.append(init)


In [80]:
pop

[array([9, 6, 1, 2, 5, 7, 4, 8, 3, 3, 2, 1, 4, 4, 3, 1, 2]),
 array([2, 6, 3, 5, 1, 9, 4, 7, 8, 2, 4, 1, 3, 4, 1, 3, 2]),
 array([6, 8, 1, 5, 2, 3, 7, 9, 4, 1, 4, 2, 3, 2, 4, 1, 3]),
 array([4, 8, 2, 9, 7, 3, 5, 6, 1, 2, 3, 1, 4, 2, 4, 3, 1]),
 array([5, 6, 1, 2, 8, 7, 9, 4, 3, 4, 2, 3, 1, 3, 2, 4, 1]),
 array([5, 2, 9, 7, 6, 4, 3, 1, 8, 2, 1, 4, 3, 1, 3, 4, 2]),
 array([8, 4, 5, 9, 2, 7, 6, 3, 1, 3, 2, 4, 1, 1, 2, 3, 4]),
 array([6, 9, 2, 7, 1, 3, 4, 5, 8, 1, 3, 2, 4, 3, 4, 1, 2]),
 array([2, 3, 5, 6, 4, 7, 1, 8, 9, 4, 2, 1, 3, 1, 2, 3, 4]),
 array([2, 6, 3, 8, 9, 4, 5, 1, 7, 1, 3, 4, 2, 3, 2, 1, 4])]

### Decode to calcuation

In [84]:
def decode_schedule(individual):
    machine_schedule = individual[:MaxOps * m].reshape((m, MaxOps))
    part_schedule = individual[MaxOps * m:MaxOps * m + n]
    assembly_schedule = individual[MaxOps * m + n:]
    
    return machine_schedule, part_schedule, assembly_schedule
for individual in pop:
    machine_schedule, part_schedule, assembly_schedule = decode_schedule(individual)
    
    # print("Machine Schedule:\n", machine_schedule)
    # print("Part Schedule:\n", part_schedule)
    # print("Assembly Schedule:\n", assembly_schedule)
print(machine_schedule)
print(part_schedule)
print(assembly_schedule)

[[2 6 3]
 [8 9 4]
 [5 1 7]]
[1 3 4 2]
[3 2 1 4]


### Fitness function

In [54]:
def decode_schedule(individual, PT, Q, M):
    n = len(Q)
    MaxOps = max(Q)
    
    # Start and end times of operations (mocked for now)
    Sp = np.random.randint(0, 20, size=(n, MaxOps))  # Start times
    Ep = Sp + np.random.randint(1, 10, size=(n, MaxOps))  # End times

    # Assembly start and end times
    Sa = np.max(Ep, axis=1) + np.random.randint(1, 5, size=n)
    Ea = Sa + np.random.randint(5, 10, size=n)

    return Sp, Ep, Sa, Ea

def fitness_function(individual, PT, Q, Ta, nd, td, no, to, nt, tt, M=1000):
    n = len(Q)       # number of parts
    MaxOps = max(Q)
    m = PT.shape[2]  # number of machines
    
    # Decode individual into scheduling
    # You need to write: decode_schedule(individual) → gives Sp, Ep, Sa, Ea, etc.
    Sp, Ep, Sa, Ea = decode_schedule(individual, PT, Q, M)
    
    E_part = np.max(Ep, axis=1)  # shape (n,)
    inventory_time = Sa - E_part
    total_inventory_time = np.sum(inventory_time)
    
    penalty_time = nd * td + no * to + nt * tt
    total_penalty_time = np.sum(penalty_time)
    
    makespan = np.max(Ea)
    
    alpha = 1.0  # weight for inventory
    beta = 1.0   # weight for penalties
    
    fitness = makespan + alpha * total_inventory_time + beta * total_penalty_time
    return fitness


In [83]:
Tp

array([[[1, 2, 8],
        [4, 8, 7],
        [6, 5, 6]],

       [[1, 3, 5],
        [7, 1, 2],
        [3, 7, 4]],

       [[5, 3, 8],
        [3, 5, 7],
        [3, 3, 2]],

       [[3, 4, 2],
        [6, 7, 2],
        [7, 8, 4]]])

In [55]:
fitness = fitness_function(individual, PT, Q, Ta, nd, td, no, to, nt, tt)
print(f"Fitness value: {fitness}")


Fitness value: 56.0
